In [1]:
import json
import pathlib
import typing

from icecream import ic
from pydantic import BaseModel
import pandas as pd

In [2]:
cve_path = pathlib.Path("cve.json")

with open(cve_path, "r") as fp:
    data = json.load(fp)

In [3]:
class Vulnerabilty (BaseModel):
    id: str
    severity: str
    description: str
    url: str
    kind: str
    artifact: str
    version: str
    fix: str
    location: typing.List[str]

In [4]:
vuln_set: dict = {}

for item in data["matches"]:
    #ic(item)
    id = item["vulnerability"]["id"]
    
    if id not in vuln_set:
        vuln_set[id] = Vulnerabilty(
            id = item["vulnerability"]["id"],
            description = item["vulnerability"]["description"],
            severity = item["vulnerability"]["severity"],
            fix = item["vulnerability"]["fix"]["versions"][0],
            url = item["vulnerability"]["urls"][0],
            kind = item["artifact"]["type"],
            artifact = item["artifact"]["name"],
            version = item["artifact"]["version"],
            location = [ item["artifact"]["locations"][0]["path"] ],
        )
    else:
        vuln_set[id].location.append(item["artifact"]["locations"][0]["path"])

In [5]:
df = pd.DataFrame([
        item.__dict__
        for item in vuln_set.values()
    ])

columns = [ "severity", "kind", "artifact", "version", "fix", "description", "location", "id", "url" ]
df = df.reindex(columns=columns)
df = df.sort_values(by=[ "severity", "kind", "artifact", "version" ])

df_report = df.drop("url", axis=1)
df_report.head()

,severity,kind,artifact,version,fix,description,location,id
4,Critical,python,Twisted,18.7.0,20.3.0,Improper Input Validation in Twisted,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-h96w-mmrf-2h6v
11,Critical,python,Twisted,18.7.0,20.3.0,HTTP Request Smuggling in Twisted,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-p5xh-vx83-mxcj
14,High,python,Django,1.11.15,2.2.25,Potential bypass of an upstream access control...,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-v6rh-hp5x-86rv
0,High,python,lxml,4.2.4,4.6.5,HTML Cleaner allows crafted and SVG embedded s...,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-55x5-fj6c-h6m8
3,High,python,pip,18.0,19.2,Path Traversal in pip,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-gpvv-69j7-gwj8


In [6]:
df_report

,severity,kind,artifact,version,fix,description,location,id
4,Critical,python,Twisted,18.7.0,20.3.0,Improper Input Validation in Twisted,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-h96w-mmrf-2h6v
11,Critical,python,Twisted,18.7.0,20.3.0,HTTP Request Smuggling in Twisted,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-p5xh-vx83-mxcj
14,High,python,Django,1.11.15,2.2.25,Potential bypass of an upstream access control...,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-v6rh-hp5x-86rv
0,High,python,lxml,4.2.4,4.6.5,HTML Cleaner allows crafted and SVG embedded s...,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-55x5-fj6c-h6m8
3,High,python,pip,18.0,19.2,Path Traversal in pip,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-gpvv-69j7-gwj8
6,High,python,py,1.5.4,1.10.0,Regular expression denial of service in py,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-hj5v-574p-mj7c
16,High,python,requests,2.19.1,2.20.0,Moderate severity vulnerability that affects r...,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-x84v-xcm2-53pg
10,High,python,urllib3,1.23,1.24.2,Improper Certificate Validation,[/home/ray/anaconda3/pkgs/conda-4.12.0-py38h06...,GHSA-mh33-7rrq-662w
13,High,python,urllib3,1.23,1.26.5,Catastrophic backtracking in URL authority par...,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-q2q7-5pp4-w6pg
9,Medium,python,Scrapy,1.5.1,1.8.1,HTTP authentication credentials potentially le...,[/home/ray/anaconda3/pkgs/conda-4.11.0-py39h06...,GHSA-jwqp-28gf-p498


In [7]:
with open(pathlib.Path("todo.json"), "w") as fp:
    fp.write(df.to_json(orient="records"))